# Relatório de Disponibilidade de Imagens de Embriões

Este relatório apresenta uma visão simplificada da saúde da pipeline e da disponibilidade de imagens, com foco especial na unidade Ibirapuera para o ano de 2025.

## 1. Carregamento de Dados

In [16]:
import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

# Caminho do banco de dados
DB_PATH = '../../../database/huntington_data_lake.duckdb'

def load_data():
    conn = duckdb.connect(DB_PATH, read_only=True)
    
    df = conn.execute("""
        SELECT 
            *,
            CAST(embryo_EmbryoDate AS DATE) as date_partition,
            STRFTIME(embryo_EmbryoDate, '%Y-%m') as month_partition,
            YEAR(CAST(embryo_EmbryoDate AS DATE)) as year_partition
        FROM silver.embryo_image_availability_latest
    """).df()
    
    conn.close()
    return df

df = load_data()
print(f"Dados carregados: {len(df):,} embriões.")

Dados carregados: 115,361 embriões.


## 2. KPIs de Alto Nível

Resumo geral da disponibilidade e distribuição de status.

In [17]:
# Calculando métricas
total = len(df)
sucesso = df[df['image_available'] == True].shape[0]
taxa_disponibilidade = (sucesso / total) * 100 if total > 0 else 0

# Mapeamento de Status para o Gráfico
def label_status(row):
    code = row['api_response_code']
    if code == 200:
        return 'Sucesso (Com Imagens)' if row['image_available'] else 'Sucesso (Sem Imagens)'
    elif code == 204:
        return 'Sem Contexto (204)'
    elif code == 500:
        return 'Erro de Servidor (500)'
    elif code == 0:
        return 'Não Verificado / Erro Rede'
    else:
        return f'Outro ({code})'

df['Status'] = df.apply(label_status, axis=1)
status_dist = df.groupby('Status').size().reset_index(name='Quantidade').sort_values('Quantidade', ascending=False)

# Visualização
fig = make_subplots(
    rows=1, cols=2, 
    specs=[[{'type':'indicator'}, {'type':'xy'}]],
    column_widths=[0.4, 0.6]
)

fig.add_trace(go.Indicator(
    mode = "gauge+number",
    value = taxa_disponibilidade,
    title = {'text': "Disponibilidade Geral (%)"},
    gauge = {'axis': {'range': [0, 100]}}, 
    domain = {'x': [0, 1], 'y': [0, 1]}
), row=1, col=1)

fig.add_trace(go.Bar(
    x=status_dist['Status'], 
    y=status_dist['Quantidade'],
    text=status_dist['Quantidade'],
    textposition='auto', 
    marker_color='royalblue'
), row=1, col=2)

fig.update_layout(title_text="Saúde da Pipeline e Disponibilidade", height=450, showlegend=False)
fig.show()

## 3. Análise por Unidade e Temporal

Desempenho por clínica e tendências mensais.

In [18]:
# Disponibilidade por Unidade
unit_stats = df.groupby('patient_unit_huntington').agg(
    total=('embryo_EmbryoID', 'count'),
    disponivel=('image_available', 'sum')
).reset_index()
unit_stats['Taxa %'] = (unit_stats['disponivel'] / unit_stats['total'] * 100).round(1)

fig_unit = px.bar(
    unit_stats.sort_values('Taxa %', ascending=False), 
    x='patient_unit_huntington', y='Taxa %', 
    color='Taxa %', text='Taxa %',
    title="Taxa de Disponibilidade por Unidade",
    labels={'patient_unit_huntington': 'Unidade', 'Taxa %': 'Disponibilidade (%)'}
)
fig_unit.update_traces(textposition='outside')
fig_unit.show()

# Tendência Mensal
monthly_stats = df.groupby(['month_partition', 'patient_unit_huntington']).agg(
    total=('embryo_EmbryoID', 'count'),
    disponivel=('image_available', 'sum')
).reset_index()
monthly_stats['Taxa %'] = (monthly_stats['disponivel'] / monthly_stats['total'] * 100).round(1)

fig_trend = px.line(
    monthly_stats, x='month_partition', y='Taxa %', color='patient_unit_huntington',
    markers=True, title="Tendência de Disponibilidade Mensal por Unidade",
    labels={'month_partition': 'Mês', 'Taxa %': 'Disponibilidade (%)', 'patient_unit_huntington': 'Unidade'}
)
fig_trend.show()

## 4. Resumo de Atenção: Ibirapuera 2025

Análise específica de falhas na unidade Ibirapuera para o ano de 2025.

In [19]:
# Filtro para Ibirapuera e 2025
ibi_2025 = df[
    (df['patient_unit_huntington'] == 'Ibirapuera') & 
    (df['year_partition'] == 2025)
].copy()

# Filtrar quem não tem dados (disponibilidade False)
sem_dados = ibi_2025[ibi_2025['image_available'] == False].copy()

print(f"--- Métricas Gerais Ibirapuera 2025 ---")
print(f"Total de embriões no período: {len(ibi_2025):,}")
print(f"Embriões sem imagens disponíveis: {len(sem_dados):,}")
print(f"Pacientes afetados: {sem_dados['patient_PatientID'].nunique():,}")

print("\n--- Tabela 1: Detalhamento por Paciente (IDs) ---")
ibi_details = sem_dados.groupby(['month_partition', 'patient_PatientID', 'treatment_TreatmentName']).agg(
    embrioes_sem_dados=('embryo_EmbryoID', 'count'),
    ultimo_erro=('error_message', 'last')
).reset_index().sort_values(['month_partition', 'embrioes_sem_dados'], ascending=[False, False])

display(ibi_details.head(10))

--- Métricas Gerais Ibirapuera 2025 ---
Total de embriões no período: 7,078
Embriões sem imagens disponíveis: 6,446
Pacientes afetados: 744

--- Tabela 1: Detalhamento por Paciente (IDs) ---


,month_partition,patient_PatientID,treatment_TreatmentName,embrioes_sem_dados,ultimo_erro
878,2025-12,885296,2025-2728,24,No images found (Empty response)
868,2025-12,874838,2025 - 2720,21,Unexpected error during data access
856,2025-12,694455,2025-2817,20,Unexpected error during data access
883,2025-12,889991,2025-2755,14,Unexpected error during data access
888,2025-12,895336,2025-2774,14,Unexpected error during data access
869,2025-12,878774,2025 - 2789,13,No images found (Empty response)
885,2025-12,893326,2025-2804,13,Unexpected error during data access
877,2025-12,884975,2025-2740,12,No images found (Empty response)
882,2025-12,888141,2025 - 2748,12,Unexpected error during data access
884,2025-12,889992,2025-2756,12,No images found (Empty response)


### Tabela Resumo Final por Mês

In [20]:
# Esta é a tabela resumida solicitada
ibi_monthly_resumo = sem_dados.groupby('month_partition').agg(
    pacientes_afetados=('patient_PatientID', 'nunique'),
    total_embrioes_sem_dados=('embryo_EmbryoID', 'count')
).reset_index().sort_values('month_partition', ascending=False)

print("Resumo por Mês (Ibirapuera 2025):")
display(ibi_monthly_resumo)

Resumo por Mês (Ibirapuera 2025):


,month_partition,pacientes_afetados,total_embrioes_sem_dados
11,2025-12,47,367
10,2025-11,98,693
9,2025-10,79,550
8,2025-09,91,593
7,2025-08,78,534
6,2025-07,75,617
5,2025-06,88,676
4,2025-05,102,726
3,2025-04,97,710
2,2025-03,81,581
